In [68]:
# pip install litellm

In [23]:
import numpy as numpy
import pandas as pd
from dotenv import load_dotenv
import os
from datetime import datetime
import random
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools import DuckDuckGoSearchResults, DuckDuckGoSearchRun
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, LiteLLMModel, tool
# from smolagents.prompts import CODE_SYSTEM_PROMPT


import smtplib


load_dotenv()

True

In [24]:
# openai_model = ChatOpenAI()
# openai_model.invoke('who is rohit sharme?')

openai_model = LiteLLMModel(model_id="gpt-4o")


In [25]:
@tool
def log_action_to_excel(action: str) -> str:
    """
    Appends a new log entry to an Excel file.

    This function checks for the existence of the Excel log file and appends a new row
    with a unique action ID, the description of the action performed, and the current timestamp.
    
    Args:
        action (str): A description of the action performed.        
    Returns:
        
    """
    
    log_file: str = "data/logs.xlsx"

    # Check if the log file already exists
    if os.path.exists(log_file):
        # Read the existing log entries
        df = pd.read_excel(log_file)
    else:
        # Create a new DataFrame with the required columns if the file doesn't exist
        df = pd.DataFrame(columns=["action_id", "action", "time_stamp"])
    
    # Generate a unique action_id by taking the max and adding 1, or starting at 1 if empty
    new_id = int(df["action_id"].max() + 1) if not df.empty else 1
    
    # Record the current timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Create a new log entry
    new_entry = {"action_id": new_id, "action": action, "time_stamp": timestamp}
    new_entry_df = pd.DataFrame([new_entry])
    
    # Append the new entry to the DataFrame
    df = pd.concat([df, new_entry_df], ignore_index=True)
    
    # Save the updated DataFrame back to the Excel file
    df.to_excel(log_file, index=False)
    
    return (f"Logged action with id {new_id}: {action}")


In [26]:
@tool
def check_item_availability(item_name: str,requested_count: int) -> str:
    """
    A tool to check the availability of an item in the warehouse.

    This function performs the following steps:
    1. Reads the shelves file ("data/shelves.xlsx") to extract the item_id associated with the given item_name.
    2. Reads the warehouse file ("data/warehouse.xlsx") to retrieve the available count for that item_id.
    3. Compares the available count to the requested_count and returns a message indicating the result.

    Args:
        item_name: The name of the item to look up.
        requested_count: The number of units requested.

    Returns:
        A string message indicating whether the item is available in the warehouse, along with the count,
        or an error message if the item cannot be found.
    """
    try:
        # Define file paths (adjust these paths as necessary)
        shelves_file = os.path.join("data", "Shelves.xlsx")
        warehouse_file = os.path.join("data", "Inventory.xlsx")
        
        # Read the shelves file to map item names to item_ids
        shelves_df = pd.read_excel(shelves_file, engine='openpyxl')
        # Perform a case-insensitive search for the item_name
        item_row = shelves_df[shelves_df['shelf_category'].str.lower() == item_name.lower()]
        if item_row.empty:
            return f"Item '{item_name}' not found in shelves."
        
        # Extract the item_id from the shelves file
        item_id = item_row.iloc[0]['item_id']
        
        # Read the warehouse file to get the available count for the item_id
        warehouse_df = pd.read_excel(warehouse_file, engine='openpyxl')
        warehouse_row = warehouse_df[warehouse_df['item_id'] == item_id]
        if warehouse_row.empty:
            return f"Item '{item_name}' (ID: {item_id}) not found in warehouse."
        
        available_count = int(warehouse_row.iloc[0]['quantity_in_warehouse'])
        
        # Compare available count with the requested count and return the result
        if available_count >= requested_count:
            return f"Item '{item_name}' (ID: {item_id}) is available. Warehouse count: {available_count}."
        else:
            return (f"Item '{item_name}' (ID: {item_id}) is available, but only {available_count} units are "
                    f"in the warehouse, which is less than the requested {requested_count}.")
    except Exception as e:
        return f"Error checking availability: {str(e)}"



In [27]:
@tool
def send_email(recipient_email: str, subject: str, body: str) -> str:
    """
    A tool to check the availability of an item in the warehouse.

    This function performs the following steps:
    1. Take the recipient_email and subject as input.
    2. Send an email to the recipient_email with the subject and body.

    Args:
        recipient_email: The email address of the recipient.
        subject: The subject of the email.
        body: The body of the email.

    Returns:
        A string message indicating whether the item is available in the warehouse, along with the count,
        or an error message if the item cannot be found.
    """
    try:
        # Fetch sender credentials from environment variables
        sender_email = os.getenv("SENDER_EMAIL")
        sender_password = os.getenv("SENDER_PASSWORD")
        if not sender_email or not sender_password:
            return "Error: Missing sender credentials in environment variables."

        # Create an SMTP session with Gmail
        with smtplib.SMTP('smtp.gmail.com', 587) as smtp:
            smtp.starttls()
            smtp.login(sender_email, sender_password)

            # Construct the raw email message with headers
            message = f"""Subject: {subject}

                        {body}
                        """
            smtp.sendmail(sender_email, recipient_email, message)

        return f"Email successfully sent to {recipient_email}."
    except Exception as e:
        return f"Failed to send email: {str(e)}"



In [28]:
@tool
def assign_worker(task: str) -> str:
    """
    Assigns a task to an available worker, updates their availability status, 
    and returns their email address for further communication.

    The Workers.xlsx file is expected to have the following columns:
      - worker_id
      - name
      - availability (e.g., "available" or "unavailable")
      - worker_email

    Args:
        task (str): A description of the work to be assigned.

    Returns:
        str: email address of the assigned worker or an error message.
    """
    try:
        # Adjust the file path as needed
        workers_file = os.path.join("data", "Workers.xlsx")

        # Read the workers data
        workers_df = pd.read_excel(workers_file, engine="openpyxl")

        # Filter to find workers whose availability is "available" (case-insensitive)
        available_workers_df = workers_df[workers_df["availability"].str.lower() == "available"]
        if available_workers_df.empty:
            return "No available workers to assign this task."

        # Select a worker from the available pool (first or random)
        # Example: pick a random worker from the available list
        assigned_worker = available_workers_df.sample(n=1).iloc[0]

        # Extract worker details
        worker_id = assigned_worker["worker_id"]
        worker_name = assigned_worker["name"]
        worker_email = assigned_worker["worker_email"]

        # Update the worker's availability to 'unavailable'
        # Use the index of the assigned worker to update the main dataframe
        workers_df.loc[assigned_worker.name, "availability"] = "Busy"

        # Save the updated DataFrame back to the Excel file
        workers_df.to_excel(workers_file, index=False)


        # Optionally, you could log this assignment using a log tool or function
        # log_action_to_excel(f"Assigned worker {worker_name} (ID: {worker_id}) to task: {task}")

        return worker_email

    except Exception as e:
        return f"Error assigning worker: {str(e)}"


In [32]:
agent = CodeAgent(tools=[],model=[])

In [76]:
project_prompt = """
Additional Project Information:
Our project is titled "AI-Powered Fulfillment Aisle Optimization & Automated Replenishment". The goal is to optimize inventory management in retail and warehouse environments by automating the process of monitoring shelf stock, triggering restocking alerts, and assigning workers or placing orders when necessary.

The system consists of (all the files are present in the data folder):
- A Workers database (Workers.xlsx) with columns: worker_id, name, availability(yes/no), worker_email.
- A Shelves database (Shelves.xlsx) with columns: shelf_id, shelf_category, maximum_capacity, current_item_count, item_id.
- A Warehouse Inventory database (Inventory.xlsx) with columns: item_id, item_name, quantity_in_warehouse, reorder_level, max_capacity.
- A Logs database (logs.xlsx) with columns: action_id, action, time_stamp.

For every task you perform, you must log the action into the logs.xlsx file.

The AI agent interacts with these Excel databases to:
- Check if shelves have sufficient items.
- Verify backup stock in the warehouse using the item_id.
- Log every action.
- Assign tasks to available workers.

**Agent Procedure:**
1. **Check Shelf Availability:**
   - Retrieve the `current_item_count` and `maximum_capacity` from the Shelves database.
   - If the shelf stock is **greater than or equal to 25%** of the maximum capacity, conclude that the stock is available and output "Stock available on the shelf."
   
2. **Handle Low Shelf Stock:**
   - If the shelf stock is **less than 25%** of the maximum capacity:
     - Check the Warehouse Inventory database for additional backup stock using the corresponding `item_id`.
     - If sufficient backup stock exists, assign an available worker (from the Workers database) the task of refilling the shelf and make sure to update the availability of the assigned worker.
     - Send an email(the email should be well formatted and must contain all necessary information) notification regarding the low stock situation and the assigned refill task.
     - Log all actions (inventory check, worker assignment, and email notification) into the Logs database.

3. **Execution Cycle:**
   - Always follow the steps in a cycle of **Thought**, **Code**, and **Observation**.
   - Ensure the correct tool arguments are used and avoid reusing variable names that conflict with tool names.
   - Remember that the state persists between code executions, so variables and imports from previous steps can be reused.
   
Do not give up; you are in charge of solving the task effectively and correctly.
"""


In [77]:
modified_system_prompt = agent.prompt_templates["system_prompt"] + project_prompt

In [78]:
# Load the default prompt_templates and then modify the system prompt.
import yaml
import importlib.resources
from smolagents import CodeAgent, LiteLLMModel, DuckDuckGoSearchTool

default_prompt_templates = yaml.safe_load(
    importlib.resources.files("smolagents.prompts").joinpath("code_agent.yaml").read_text()
)

# Modify the system prompt as needed.
default_prompt_templates["system_prompt"] = modified_system_prompt

# Create the agent without passing system_prompt directly.
agent = CodeAgent(
    tools=[send_email],
    model=LiteLLMModel(model_id="gpt-4o"),
    additional_authorized_imports=["pandas", "numpy", "smtplib", "datetime", "os", 'openpyxl'],
    prompt_templates=default_prompt_templates
)


In [79]:
# print(agent.system_prompt)

In [80]:
result = agent.run("Check the availability of peanut butter ")


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Check the availability of peanut butter                                                                         │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o ─────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Load the Shelves database                                                                                      
  shelves_df = pd.read_excel('data/Shelves.xlsx')                                                                  
                                                                                                                   
  # Find the item_id for peanut butter                                                                             
  peanut_butter_row = shelves_df[shelves_df['shelf_category'].str.contains('peanut butter', case=False,            
  na=False)]                                                                                                       
  peanut_butter_id = peanut_butter_row['item_id'].values                                                           
                                                                                                                   
  print(f'Peanut Butter Item ID: {peanut_butter_id}')                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Peanut Butter Item ID: [101]

Out: None

[Step 1: Duration 3.91 seconds| Input tokens: 4,461 | Output tokens: 226]

...

[Step 8: Duration 9.16 seconds| Input tokens: 45,423 | Output tokens: 1,467]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 9 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Peanut butter stock check and replenishment task completed successfully.")                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Peanut butter stock check and replenishment task completed successfully.

[Step 9: Duration 15.64 seconds| Input tokens: 53,036 | Output tokens: 1,551]

In [55]:
result

'Peanut butter availability checked. Current stock in warehouse: 5. Reorder required as stock is below reorder level.'